In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import numpy as np

In [3]:
caminho_base = 'dados/'

In [5]:
# Gerador de imagens para o treinamento
gerador_treino = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.15
)

# Gerador para treino e validação
dados_treino = gerador_treino.flow_from_directory(
    caminho_base,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

dados_validacao = gerador_treino.flow_from_directory(
    caminho_base,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Carregar o modelo ResNet50 pré-treinado
base_modelo = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Found 1530 images belonging to 17 classes.
Found 270 images belonging to 17 classes.
94765736/94765736 [==============================] - 4s 0us/step


In [9]:
# Congelar as camadas base
for camada in base_modelo.layers:
    camada.trainable = False

# Adicionar camadas customizadas para classificação
x = base_modelo.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
saida = Dense(17, activation='softmax')(x)

modelo = Model(inputs=base_modelo.input, outputs=saida)

In [10]:
# Compilar o modelo
modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Treinar o modelo
historico = modelo.fit(
    dados_treino,
    steps_per_epoch=dados_treino.samples // 32,
    validation_data=dados_validacao,
    validation_steps=dados_validacao.samples // 32,
    epochs=10
)

Epoch 1/10
47/47 [==============================] - 88s 2s/step - loss: 3.0197 - accuracy: 0.0814 - val_loss: 2.8687 - val_accuracy: 0.0547
Epoch 2/10
47/47 [==============================] - 80s 2s/step - loss: 2.8352 - accuracy: 0.1008 - val_loss: 2.7769 - val_accuracy: 0.1562
Epoch 3/10
47/47 [==============================] - 80s 2s/step - loss: 2.8096 - accuracy: 0.1081 - val_loss: 2.8102 - val_accuracy: 0.0625
Epoch 4/10
47/47 [==============================] - 81s 2s/step - loss: 2.7811 - accuracy: 0.1108 - val_loss: 2.7701 - val_accuracy: 0.1016
Epoch 5/10
47/47 [==============================] - 78s 2s/step - loss: 2.7797 - accuracy: 0.1121 - val_loss: 2.7721 - val_accuracy: 0.1250
Epoch 6/10
47/47 [==============================] - 78s 2s/step - loss: 2.7708 - accuracy: 0.1255 - val_loss: 2.7024 - val_accuracy: 0.1406
Epoch 7/10
47/47 [==============================] - 80s 2s/step - loss: 2.7397 - accuracy: 0.1228 - val_loss: 2.7133 - val_accuracy: 0.1406
Epoch 8/10
47/47 [==

In [13]:
# Função para prever o artista com uma nova imagem
def prever_artista(caminho_imagem):
    imagem = image.load_img(caminho_imagem, target_size=(224, 224))
    img_array = image.img_to_array(imagem) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predicao = modelo.predict(img_array)
    indice = np.argmax(predicao)
    return dados_treino.class_indices, indice

In [15]:
# Exemplo de uso
caminho_imagem_nova = f'{caminho_base}Will Smith/001_beebcee2.jpg'
classes, predicao = prever_artista(caminho_imagem_nova)
artista_predito = list(classes.keys())[list(classes.values()).index(predicao)]

print(f'O artista predito é: {artista_predito}')

1/1 [==============================] - 0s 114ms/step
O artista predito é: Tom Hanks


In [ ]:
print(f'O artista predito é: {artista_predito}')